In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Load data
data = pd.read_csv('data.csv')

# Define features and target variables
X = data.drop(['respondent_id', 'xyz_vaccine', 'seasonal_vaccine'], axis=1)
y_xyz = data['xyz_vaccine']
y_seasonal = data['seasonal_vaccine']

# Identify numeric and categorical columns
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Preprocessing pipeline for numeric and categorical data
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Preprocess the data
X_processed = preprocessor.fit_transform(X)




In [8]:
# Split the data
X_train, X_test, y_train_xyz, y_test_xyz = train_test_split(X_processed, y_xyz, test_size=0.2, random_state=42)
_, _, y_train_seasonal, y_test_seasonal = train_test_split(X_processed, y_seasonal, test_size=0.2, random_state=42)


In [10]:
from sklearn.linear_model import LogisticRegression

# Train model for xyz_vaccine
model_xyz = LogisticRegression(max_iter=5000)
model_xyz.fit(X_train, y_train_xyz)

# Train model for seasonal_vaccine
model_seasonal = LogisticRegression(max_iter=5000)
model_seasonal.fit(X_train, y_train_seasonal)


LogisticRegression(max_iter=5000)

In [11]:
from sklearn.metrics import roc_auc_score

# Predict probabilities
y_pred_prob_xyz = model_xyz.predict_proba(X_test)[:, 1]
y_pred_prob_seasonal = model_seasonal.predict_proba(X_test)[:, 1]

# Calculate ROC AUC
roc_auc_xyz = roc_auc_score(y_test_xyz, y_pred_prob_xyz)
roc_auc_seasonal = roc_auc_score(y_test_seasonal, y_pred_prob_seasonal)

# Mean ROC AUC
mean_roc_auc = (roc_auc_xyz + roc_auc_seasonal) / 2
print(f'Mean ROC AUC: {mean_roc_auc}')


Mean ROC AUC: 0.8437125737041244


In [16]:
import pandas as pd

# Load test data
test_data = pd.read_csv('test_set_features.csv')

# Preprocess the test data using the same preprocessor used for the training data
X_test_processed = preprocessor.transform(test_data)

# Predict probabilities on the test dataset
y_pred_prob_xyz_full = model_xyz.predict_proba(X_test_processed)[:, 1]
y_pred_prob_seasonal_full = model_seasonal.predict_proba(X_test_processed)[:, 1]

# Prepare the submission file
submission = pd.DataFrame({
    'respondent_id': test_data['respondent_id'],
    'xyz_vaccine': y_pred_prob_xyz_full,
    'seasonal_vaccine': y_pred_prob_seasonal_full
})

# Save the submission file
submission.to_csv('submissionfile.csv', index=False)
